# *Music Scheduling using Evolutionary Algorithms*
This notebook implements a music scheduling algorithm for generating playlists which have an exact length (e.g. 60 minutes) while respecting constraints, i.e. specific categories of playlist elements. This could be used to create playlists for radio stations which need to backtime to a specific point of time.

## General
For using the algorithm, different data is required.
- playlistelements.csv
- playliststructure.csv

### playlistelements.csv
The playlistelements.csv contains information about elements which can be placed into the playlist. More specifically, this is the length of the element in seconds, the category of the element and a unique identifier.

### playliststructure.csv
The playliststructure.csv maintains information about the desired playlist structure, i.e. which element has to be played at a specific point of time.

# _Data Generation_

## Retrieve playlistelements.csv
For now, we are going to retrieve a PlaylistElements from the [CORGIS Dataset Project](https://think.cs.vt.edu/corgis/csv/music/music.html). We download the original file and create a new csv file which contains only the necessary data.

### Download File

In [1]:
# Imports
import urllib
import urllib.request

# Download file
urllib.request.urlretrieve('https://think.cs.vt.edu/corgis/csv/music/music.csv?forcedownload=1', 'playlistelementstmp.csv')

('playlistelementstmp.csv', <http.client.HTTPMessage at 0x20d54bb9550>)

### Clean File / Create cleaned File

In [2]:
# Imports
import csv

# Genre list. later used for playliststructure.csv
genres = []

def getgenre(fieldname):
    if len(fieldname) > 0 and ' ' in fieldname:
            return fieldname.split(' ')[0]
    return fieldname

fieldnames = ['artist', 'song', 'duration', 'genre']
with open('playlistelementstmp.csv', newline='') as readcsv, open('playlistelements.csv', 'w', newline='') as writecsv:
        csvreader = csv.DictReader(readcsv, delimiter=',')
        csvwriter = csv.DictWriter(writecsv, delimiter=',', fieldnames=fieldnames)
        csvwriter.writeheader()
        for i, row in enumerate(csvreader):
            genre = getgenre(row['artist_mbtags'])
            csvwriter.writerow({'artist': row['artist.name'],
                                'song': row['title'],
                                'duration': round(float(row['duration'])),
                                'genre': genre})
            if not(genre in genres) and len(genre) > 0:
                genres.append(genre)
        print('Tranformed and wrote', i, 'rows and found', len(genres), 'genres')

Tranformed and wrote 9999 rows and found 241 genres


## Create a playliststructure.csv

In [3]:
# Imports
import random

# Set seed for reproducability
random.seed(1)

# Number of elements
NO_ELEMENTS = 20
# Probability of 'any' element (== Any Element can be played)
PROB_ANY = 0.45

fieldnames = ['element_categorie']
with open('playliststructure.csv', 'w', newline='') as writecsv:
    csvwriter = csv.DictWriter(writecsv, delimiter=',', fieldnames=fieldnames)
    csvwriter.writeheader()
    for i in range(NO_ELEMENTS):
        if random.random() <= PROB_ANY:
            element_categorie = 'ANY'
        else:
            element_categorie = genres[random.randrange(0, len(genres))]
        csvwriter.writerow({'element_categorie': element_categorie})

# _Playlist Generation_

Now this is where things get interesting: We are gonna create a playlist with evolutionary algorithms. Therefore we first load the elements and the playliststructure.

## Read CSV Files

In [4]:
# Imports

import csv
import random

playlist_structure = []
playlist_elements = []

with open('playliststructure.csv', 'r', newline='') as readstructure:
    structurereader = csv.DictReader(readstructure, delimiter=',')
    for r in structurereader:
        playlist_structure.append(r['element_categorie'])
        
with open('playlistelements.csv', 'r', newline='') as readelements:
    elementsreader = csv.DictReader(readelements, delimiter=',')
    for r in elementsreader:
        playlist_elements.append(r)

## Define Evolutionary Algorithms
**General**
This is subset sum problem, since we want to find a subset of the songs which are - in sum - exactly 60 Minutes long. Following wikipedia, this problem is [np-complete](https://en.wikipedia.org/wiki/Subset_sum_problem).

### Generate Helping Functions


In [5]:
# Permutation changes one item of the solution to another one
def mutation(solution, probability = 0.05):
    for i in range(len(solution)):
        if random.random() <= probability:
            # Do permutation
            newitem = random.randrange(0, len(playlist_elements))
            while True:
                newitem = random.randrange(0, len(playlist_elements))
                if not(newitem in solution):
                    break
            solution[i] = newitem
    return solution

def crossover(solution0, solution1):
    solution0 = sorted(solution0, key=lambda x:int(playlist_elements[x]['duration']))
    solution1 = sorted(solution1, key=lambda x:int(playlist_elements[x]['duration']))
    cutpoint = len(solution0) // 2
    child = solution0[0:cutpoint]
    for x in solution1:
        if not(x in child):
            child.append(x)
    return child

def generatechild(length):
    solution = []
    for i in range(length):
        while True:
            v = random.randrange(0, len(playlist_elements))
            if not(v in solution):
                break
        solution.append(v)
    return solution

def evaluate(solution, maxlength):
    playlist_length = [int(x['duration']) for x in [playlist_elements[p] for p in solution]]
    return abs(maxlength-sum(playlist_length))

### EA

In [6]:
MAX_SUM = 60 * 60 #60 Seconds * 60 Minutes = 3600 Seconds
SOL_LENGTH = len(playlist_structure)
GEN_SIZE = 100 # Amount of child
MAX_EVAL = 100 # Maximum Evaluations

solutions = [generatechild(SOL_LENGTH) for i in range(GEN_SIZE)]

solution_evaluation = [evaluate(x, MAX_SUM) for x in solutions]
sorted_indices = sorted(range(len(solution_evaluation)),key=lambda x:solution_evaluation[x])
    
for iteration in range(MAX_EVAL):
    for i in range(GEN_SIZE):
        #For now: Just take parents from the best 10%. --> Maybe change later!
        #This differs strongly from general EAs, but for this task it is suitable.
        parent1 = solutions[random.randrange(0, int(len(solutions) * .1))]
        parent2 = solutions[random.randrange(0, int(len(solutions) * .1))]
        co_child = crossover(parent1[:], parent2[:])
        co_child = mutation(co_child, 0.1)
        solutions.append(co_child)

    solution_evaluation = [evaluate(x, MAX_SUM) for x in solutions]
    sorted_indices = sorted(range(len(solution_evaluation)),key=lambda x:solution_evaluation[x])
    #if iteration % 10 == 0:
    print('Generation', iteration, '-', 'Best Value -', solution_evaluation[sorted_indices[0]])
    solutions = [solutions[x] for x in sorted_indices[0:GEN_SIZE]]
    if solution_evaluation[sorted_indices[0]] == 0:
        break
print('Best Solution:', solutions[0])

Generation 0 - Best Value - 106
Generation 1 - Best Value - 106
Generation 2 - Best Value - 106
Generation 3 - Best Value - 106
Generation 4 - Best Value - 106
Generation 5 - Best Value - 106
Generation 6 - Best Value - 7
Generation 7 - Best Value - 7
Generation 8 - Best Value - 7
Generation 9 - Best Value - 1
Generation 10 - Best Value - 1
Generation 11 - Best Value - 1
Generation 12 - Best Value - 1
Generation 13 - Best Value - 1
Generation 14 - Best Value - 1
Generation 15 - Best Value - 1
Generation 16 - Best Value - 0
Best Solution: [5892, 2145, 5198, 6949, 5972, 2299, 157, 3579, 9873, 1482, 4492, 213, 1475, 9545, 8295, 7277, 593, 3807, 6035, 9381]
